<h1><center>Big Data Algorithms Techniques & Platforms</center></h1>

<h2>
<hr style=" border:none; height:3px;">
<center>Lab 2: Introduction to Spark</center>
<hr style=" border:none; height:3px;">
</h2>

# A. Introduction


<p align="justify">
<font size="3">
In this little set of exercises you'll learn basic Spark programming skills that are necessary to develop simple, yet powerful, applications to be executed in a distributed environment.
</font>
</p>

<p align="justify">
<font size="3">
The assignment is presented in this __Jupyter Notebook__, an interface that offers support for text, code, images and other media. Essentially, a Jupyter Notebook consists of multiple _cells_, either containing some text, like the one that you are reading, or code that you can execute.
</font>
</p>

<p align="justify">
<font size="3">
The cells that contain code are marked with the label "In [  ]" that is found on the left of the cell. In order to execute the code in the cell, you need to select the cell and press Shift+Enter (hold Shift while pressing Enter). During the execution of the code, the label on the left of the cell will change to "In [ * ]"; when the execution is over, the asterisk is replaced by a number.
</font>  
    
<font size="2">
IMPORTANT: wait for the execution of the cell to be over before proceeding in the Notebook.
</font>
<font size="3">
Whenever you define a variable or a function in a cell, that variable and function will be visible in the cells below.
This means that one can split the code of an application across different cells to interleave it with textual explanations.
    </font>
</p>

<p align="justify">
<font size="3">
Spark supports four programming languages: Scala (the one used to implement Spark itself), Java, R and Python. In this assignment we use Python.
The assignment will guide you through the Spark programming notions by using simple examples.
After the examples, the exercises will give you the chance to practice those notions.
</font>
</p>


<p align="justify">
<font size="3">
**Apache Spark** is a cluster computing framework for parallel data processing that was conceived to address the inefficiencies of Hadoop with respect to iterative computations.
Spark is used by both data scientists, who analyze large datasets, and engineers, who develop data processing applications. Spark allows both to concentrate on their application by hiding all the complexity of running applications in a distributed environment: distributed systems programming, network communication and fault tolerance.
</font>
</p>

<p align="justify">
<font size="3">
In order to run a distributed computation on Spark, one has to develop a **Spark application**.
A Spark application runs as a set of independent processes (called the _Executors_) across the machines (a.k.a., _Worker_ nodes) of a cluster, coordinated by the _Driver_, the process that runs the $main()$ function of the application.
</font>
</p>

<p align="justify">
<font size="3">
The _Driver_ creates an object called _SparkContext_ that communicates with the underlying cluster manager and coordinates the distribution of the computation across the _Executors_.
For example, if we were running an application to count the number of lines in a file,
different machines might count lines in different ranges of the file.
</font>
</p>

<p align="justify">
<hr style=" border:none; height:2px;">
 <font  size="3" color='#91053d'>**Execute the following cell in order to initialize the _SparkContext_.**</font>
<hr style=" border:none; height:2px;">
</p>

In [1]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://downloads.apache.org/spark/spark-3.5.0/spark-3.5.0-bin-hadoop3.tgz
!tar zxvf spark-3.5.0-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.0-bin-hadoop3"

import findspark
findspark.init()
from pyspark import SparkConf, SparkContext
conf = SparkConf().setMaster("local")
sc = SparkContext(conf = conf)
print("initialization successful!")

import numpy as np
import random as rn

seed_value=0
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:7 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,016 kB]
Get:8 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease [18.1 kB]
Get:9 http://security.ubuntu.com/ubuntu jammy-security/multiverse amd64 Packages [44.0 kB]
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,144 kB]
Get:11 http://archive.ubuntu.com/ubuntu jammy-updates/multiverse amd64 Packages [49.8 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,16

In [2]:
!apt-get install unzip
!unzip data_spark.zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.
Archive:  data_spark.zip
   creating: data/
  inflating: data/.DS_Store          
   creating: __MACOSX/
   creating: __MACOSX/data/
  inflating: __MACOSX/data/._.DS_Store  
  inflating: data/neo4j.log          
  inflating: __MACOSX/data/._neo4j.log  
  inflating: data/moby-dick.txt      
  inflating: __MACOSX/data/._moby-dick.txt  
  inflating: data/stopwords.txt      
  inflating: __MACOSX/data/._stopwords.txt  
  inflating: __MACOSX/._data         


## A.1 Resilient Distributed Dataset (RDD)


<p align="justify">
<font size="3">
Spark distributes the data and the computations across the machines of a cluster by using the notion of **Resilient Distributed Dataset (RDD)**.
An RDD is an **immutable** distributed collection of data.
Each element of an RDD can be an instance of any Python type, including a user-defined class.
The _SparkContext_  splits an RDD into multiple _partitions_ and scatters them across different machines of the cluster.
</font>
</p>

<p align="justify">
<font size="3">
The distribution of the partitions of an RDD is completely transparent to the application.
The only thing a Spark application has to do is to create some RDDs and
specify the computations on these RDDs, by using special functions that Spark provides to this purpose.
</font>
</p>

### Creating RDDs

<p align="justify">
<font size="3">
Spark provides two ways to create an RDD:

<ul>
<li> By distributing a collection of objects.
<li> By loading an external dataset (either in a file or a database).
</ul>
</font>
</p>

<hr style="border:2px solid;">

<p>
<font size="3" color='#91053d'><strong>Execute the following code to create an RDD called $words$, where each element is a string taken from the list $wordList$.</strong></font>
</p>
<hr style="border:2px solid;">

In [3]:
wordList = ["Al", "Ani", "Jackie", "Lalitha", "Mark", "Neil", "Nick", "Shirin", "Jackie", "Mark", "Ani", "Mark"]
words = sc.parallelize(wordList)
print("Done!")

Done!


<p align="justify">
<font size="3">
Once created, Spark provides two types of  operations on a RDD:
<ol>
<li> **Transformations**. A transformation takes in one or more RDDs and returns a new RDD.
<li> **Actions**. An action takes in an RDD and outputs a value.
</ol>
</font>
</p>

<p>
</p>

<p align="justify">
<font size="3">
One common transformation is filtering data that matches a predicate by using the function $filter$.
The function $filter$ is applied on an RDD and takes in a predicate.
It loops through each element of the RDD and verifies whether that element satisfies the predicate. The function $filter$ outputs a new RDD whose elements are those that satisfy the predicate.
</font>
</p>

<hr style=" border:2px solid;">
<p>

 <font size="3" color='#91053d'>**Execute the following code to create an RDD called $nNames$ by retaining only the names whose first letter is 'N' from the RDD $words$ .**</font>

</p>
<hr style=" border:2px solid;">

In [4]:
'''
As mentioned before, the function filter takes in a predicate that is itself a function (returning a boolean value).
An elegant way to pass a function to a function in Python is the lambda notation.

In the code below, the argument of the function filter is a function that takes in a variable called
"name"; the type of this variable must match the type of the elements of the RDD words, in this case
a string.
Then the function returns whether the first character of the string "name" is N.

Another way to express the same thing without the lambda notation would be to explicitly define the predicate,
as follows:

def predicate(name):
    return name[:1]=='N'

nNames = words.filter(predicate)

'''
nNames = words.filter(lambda name: name[:1]=='N')
print("done!")

done!


<p align="justify">
<font size="3">
The transformation $filter$ above is not executed by Spark until an action is called on the RDD $nNames$.
In general, Spark postpones the execution of a transformation on a RDD to when an action is invoked on the RDD itself. This is called **lazy evaluation**. The reason for this approach will be clearer in the next example.
    </font>
</p>

<p align="justify">
<font size="3">
One example of action is the function $first()$ that returns the first element in the RDD.
    </font>
</p>

<hr style=" border:2px solid;">
<p>
 <font size="3" color='#91053d'><strong>Execute the following code to print the first element of the RDD $nNames$.</strong></font>
</p>
<hr style=" border:2px solid;">

In [5]:
'''
REMEMBER: the variable nNames has been defined in the cell above, so
it is VISIBLE in this cell as well as in the cells below the current one
'''
print(nNames.first())

Neil


<p align="justify">
<font size="3">
The previous example clearly shows that a Spark application is essentially a sequence of operations that create, transform and perform some actions on RDDs.
</font>
</p>

<p align="justify">
<font size="3">
The following code shows an example of creation of RDD from an external dataset, more precisely a text file that contains log information of a Neo4j database.
The function $textFile()$ takes in the path to the input text file and returns an RDD, where each element is a line of the file.
The code goes through the following steps:

<ol>
<li> **RDD creation**. Creates a RDD called $lines$, where each element is a line from the input text file.
<li> **Transformation filter**. Creates a RDD called $exceptions$ from the RDD $lines$ by only retaining the lines containing the string "exception".
<li> **Action count()**. Counts the number of elements of the RDD $exceptions$.
<li> **Action take**. Prints the first line of the RDD $exceptions$.
</ol>
</font>
</p>

<p>
</p>

<p align="justify">
<font size="3">
Here we can see the advantage of the _lazy evaluation_ of transformations.
Spark does not execute the function $textFile()$ as soon as it is invoked, which would result
in loading into main memory the whole content of the log file (it can be very large).
Instead, it waits until the first action $count()$ is invoked.
Since this action is called on a filtered version of the RDD $lines$,
Spark will load into memory only the lines that contain the word "exception".
</font>
</p>

<hr style=" border:2px solid;">
<p>
 <font size="3" color='#91053d'><strong>Execute the following code.</strong></font>
</p>
<hr style=" border:2px solid;">

In [6]:
data_file = "./data/neo4j.log"

# 1. RDD creation
lines = sc.textFile(data_file)

#2. RDD filter
exceptions = lines.filter(lambda line : "exception" in line)

#3. Action count()
nbLines = exceptions.count()
print("Number of exception lines ", nbLines)

'''
IMPORTANT: The output shows the first line of the log file that contains
words such as "ERROR" and "failed".
This does not mean that Spark is raising an error;
it is just showing the content of the input file.
So everything's fine :)
'''

#4. Print first line.
exceptions.take(1)

#Here the text will visualize the content of the log file that contain a text starting with an error code:

#['2017-10-06 13:12:26.097+0000 ERROR Failed to start Neo4j: Starting Neo4j failed: Component \'org.neo4j.server.database.
#LifecycleManagingDatabase@2814f71a\' was successfully initialized, but failed to start. Please see the attached c ...



Number of exception lines  5


['2017-10-06 13:12:26.097+0000 ERROR Failed to start Neo4j: Starting Neo4j failed: Component \'org.neo4j.server.database.LifecycleManagingDatabase@2814f71a\' was successfully initialized, but failed to start. Please see the attached cause exception "Format version is not supported (resource BufferedChecksumIndexInput(MMapIndexInput(path="/Users/quercini_gia/Documents/software/neo4j-community-3.2.5/data/databases/social-network/upgrade/index/lucene/relationship/crosslinks/segments_1"))): -11 (needs to be between 1071082519 and 1071082519). This version of Lucene only supports indexes created with release 4.0 and later.". Starting Neo4j failed: Component \'org.neo4j.server.database.LifecycleManagingDatabase@2814f71a\' was successfully initialized, but failed to start. Please see the attached cause exception "Format version is not supported (resource BufferedChecksumIndexInput(MMapIndexInput(path="/Users/quercini_gia/Documents/software/neo4j-community-3.2.5/data/databases/social-network/u

## A.2 Transformations

<p align="justify">
<font size="3">
Here are some common transformations in Spark.
In the following list, $r$ denotes the RDD on which the transformation is invoked.
<ol>
<li> $r.filter(pred)$. Returns a RDD consisting of only the elements of the input RDD $r$ that satisfy the given predicate.
<li> $r.map(func)$. Applies a function $func$ to each element of the input RDD $r$ and returns an RDD of the result.
<li> $r.flatMap(func)$. Same as $map()$, but used when $map()$ would return a RDD where each element is a list.
<li> $r.union(other)$. Takes in two RDDs ($r$ and $other$) and returns a RDD that contains the elements from both. Unlike the mathematical operation, $union$ in Spark does not remove the duplicates.
<li> $r.intersection(other)$. Takes in two RDDs ($r$ and $other$) and returns a RDD that contains the elements found in both.
<li> $r.subtract(other)$. Takes in two RDDs ($r$ and $other$) and returns a RDD that contains the elements from the RDD $r$, except those that are found in $other$.
<li> $r.cartesian(other)$. Takes in two RDDs ($r$ and $other$) and returns a RDD that contains the Cartesian product of both.
<li> $r.distinct()$. Returns a RDD that contains the same elements as the input RDD $r$ without duplicates.
</ol>
    
We are now going to look at an example of use of these transformations.
</font>    
</p>

<hr style=" border:2px solid;">
<p>
 <font size="3" color='#91053d'><strong>Execute the following code to create the two RDDs $r1$ and $r2$</strong></font>
</p>
<hr style=" border:2px solid;">

In [7]:
r1 = sc.parallelize([1, 2, 3, 4])
r2 = sc.parallelize([3, 4, 5, 6, 7])
print("done")

done


### Use of $map()$

<p align="justify">
<font size="3">
Here we see an example of the transformation $map()$.
</font>
</p>

<hr style=" border:2px solid;">
<p>
<font size="3" color='#91053d'>**Execute the following code to create:
 <ol>
 <li> a RDD $square$, where each element is the square of the corresponding element in $r1$;
 <li> a RDD $half$, where each element is the half of the corresponding element in $r2$.**
 </ol>
 </font>
</p>
<hr style=" border:2px solid;">

In [8]:
square = r1.map(lambda x : x*x)
half = r2.map(lambda x: x/2)

# The function collect() is an action that transforms the RDD into a Python list that can be printed.
print("Elements of RDD square ", square.collect())
print("Elements of the RDD half ", half.collect())

Elements of RDD square  [1, 4, 9, 16]
Elements of the RDD half  [1.5, 2.0, 2.5, 3.0, 3.5]


### Use of $flatMap()$

<p align="justify">
<font size="3">
The transformation $flatMap()$ works by applying the transformation $map()$ on the input RDD;
if each element of the resulting RDD is a list, $flatMap()$ returns a RDD where all lists are merged.
In other words, when $map()$ returns a RDD where each element is a list, $flatMap()$ returns a RDD where each element is a value of that list.
</font>
</p>

<p align="justify">
<font size="3">
Let's see an example. Suppose that we want to return a RDD from $r1$ where each element is associated to its square.
More precisely, the output RDD will be as follows:
<center>
[ [1, 1], [2, 4], [3, 9], [4, 16] ]
</center>
</font>    
</p>

<hr style=" border:solid 2px;">
<p>

 <font size="3" color='#91053d'>**Execute the following code to create a RDD $squares$ where each element of $r1$ is associated to its square.**</font>
</p>
<hr style=" border:solid 2px;">

In [9]:
squares = r1.map(lambda x : [x, x*x])
print("Elements of RDD squares ", squares.collect())

Elements of RDD squares  [[1, 1], [2, 4], [3, 9], [4, 16]]


<p align="justify">
<font size="3">

As you can see, each element of the RDD $squares$ is a list of two elements; indeed,
after calling the action $collect()$, we obtain a list of lists in Python.
If, instead, we want a simple list of elements, we have to invoke $flatMap()$ on $r1$.
More precisely, with $flatMap()$ we obtain the following RDD:
<center>
[1, 1, 2, 4, 3, 9, 4, 16]
</center>
</font>
</p>

<hr style=" border:solid 2px;">
<p>

 <font size="3" color='#91053d'><strong>Execute the following code to create an RDD $squares$ where each element of $r1$ is associated to its square (but each element is just a value instead of a list of values</strong></font>
</p>
<hr style=" border:solid 2px;">

In [10]:
squares = r1.flatMap(lambda x : [x, x*x])
print("Elements of RDD squares ", squares.collect())

Elements of RDD squares  [1, 1, 2, 4, 3, 9, 4, 16]


### Use of set transformations

<p align="justify">
<font size="3">
The set transformations are $union$, $intersection$, $subtract$ and $cartesian$.
</font>
</p>

<hr style=" border:solid 2px;">
<p>
 <font size="3" color='#91053d'>**Execute the following code to see an example of these transformations**</font>
</p>
<hr style=" border:solid 2px;">

In [11]:
union = r1.union(r2)
intersection = r1.intersection(r2)
subtract = r1.subtract(r2)
cartesian = r1.cartesian(r2)

print("Elements of RDD r1 ", r1.collect())
print("Elements of RDD r2 ", r2.collect())
print("Elements of RDD union ", union.collect())
print("Elements of RDD intesection ", intersection.collect())
print("Elements of RDD subtract ", subtract.collect())
print("Elements of RDD cartesian ", cartesian.collect())


Elements of RDD r1  [1, 2, 3, 4]
Elements of RDD r2  [3, 4, 5, 6, 7]
Elements of RDD union  [1, 2, 3, 4, 3, 4, 5, 6, 7]
Elements of RDD intesection  [4, 3]
Elements of RDD subtract  [2, 1]
Elements of RDD cartesian  [(1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (3, 3), (3, 4), (3, 5), (3, 6), (3, 7), (4, 3), (4, 4), (4, 5), (4, 6), (4, 7)]


### Use of $distinct()$

<p align="justify">
<font size="3">
The transformation $distinct()$ returns an RDD that contains the same elements as the input RDD without the duplicates.
</font>
</p>

<hr style=" border:solid 2px;">
<p>
<font size="3" color='#91053d'><strong>Execute the following code to see an example of use of $distinct()$</strong></font>
</p>
<hr style=" border:solid 2px;">

In [12]:
nodup = union.distinct()
print("Elements of the RDD union: ", union.collect())
print("Elements of the RDD union (with no duplicates): ", nodup.collect())

Elements of the RDD union:  [1, 2, 3, 4, 3, 4, 5, 6, 7]
Elements of the RDD union (with no duplicates):  [2, 4, 6, 1, 3, 5, 7]


## A.3 Actions

<p align="justify">
<font size="3">
Here are some common actions in Spark.
As with transformations, $r$ denotes the RDD on which the action is invoked.

<ol>
<li> $r.reduce(func)$. Performs a pair-wise application of the given function $func$ to the elements of the input RDD $r$.
<li> $r.collect()$. Returns a Python list with all the elements of the input RDD $r$.
<li> $r.count()$. Returns the number of elements in the input RDD $r$.
<li> $r.countByValue()$. Returns the number of times each element occurs in the input RDD $r$.
<li> $r.take(num)$. Prints the first $num$ elements of the input RDD $r$.
<li> $r.top(num)$. Prints the top $num$ elements of the input RDD $r$ (sorted in decreasing order).
</ol>
</font>
</p>

### Use of $reduce(func)$

<p align="justify">
<font size="3">
The action $reduce(func)$ performs a pair-wise application of the given function $func$ on the elements of the input RDD.
</font>
</p>

<hr style=" border:solid 2px;">
<p>
<font size="3" color='#91053d'><strong>Execute the following code to sum all values of the RDD $r1$</strong></font>
</p>
<hr style=" border:solid 2px;">


In [13]:
'''
The function passed to the reduce MUST take in two arguments
that have the same type as the elements of the input RDD
'''
sum = r1.reduce(lambda x, y : x + y)
print("Elements of the RDD r1 ", r1.collect())
print("Sum of the elements of the RDD r1: ", sum)


Elements of the RDD r1  [1, 2, 3, 4]
Sum of the elements of the RDD r1:  10


### Use of $countByValue()$

<p align="justify">
<font size="3">
The action $countByValue()$ counts the number of the occurrences of each element of the input RDD.
The result is a Python dictionary, where a key is an element of the input RDD and the corresponding value the number of its occurrences.
</font>
</p>

<hr style=" border:solid 2px;">
<p>

<font size="3" color='#91053d'>**Execute the following code to get the number of occurrences of each element in the RDD $union$**</font>

</p>
<hr style=" border:solid 2px;">

In [14]:
occurrences = union.countByValue()
print("Elements of the RDD union ", union.collect())
print("Occurrences of each element in the RDD union:")
for k, v in occurrences.items():
    print(k, " --> ",  v, " occurrences")

Elements of the RDD union  [1, 2, 3, 4, 3, 4, 5, 6, 7]
Occurrences of each element in the RDD union:
1  -->  1  occurrences
2  -->  1  occurrences
3  -->  2  occurrences
4  -->  2  occurrences
5  -->  1  occurrences
6  -->  1  occurrences
7  -->  1  occurrences


# B Pair RDDs

<p align="justify">
<font size="3">
Pair RDDs are simply RDDs where each element is a key-value pair.
</font>
</p>

<p align="justify">
<font size="3">
We are now going to look at examples of  transformations and actions on pair RDDs.
</font>
</p>

## B.1 Creation of Pair RDDs

<p align="justify">
<font size="3">
One common way to create a pair RDD is to transform an existing RDD with a $map()$.
</font>
</p>

<hr style=" border:solid 2px;">
<p align="justify">
<font size="3" color='#91053d'><strong>Execute the following code to create a Pair RDD $kvwords$ from the RDD $words$. Each element of $kvwords$ is a pair where the key is a word and the value is 1</strong></font>
</p>
<hr style=" border:solid 2px;">

In [15]:
kvwords = words.map(lambda word : (word, 1))
print(kvwords.take(5))

[('Al', 1), ('Ani', 1), ('Jackie', 1), ('Lalitha', 1), ('Mark', 1)]


## B.2 Transformations on Pair RDDs


<p align="justify">
<font size="3">
All transformations applied to standard RDDs can be applied to Pair RDDs as well. The only difference is that any function that is passed to a transformation or an action must take in pairs instead of single values.
</font>
</p>

<p align="justify">
<font size="3">
In addition, the following transformations can **only** be applied to Pair RDDs:
<ol>
<li> $r.reduceByKey(func)$. It applies the given function $func$ pairwise to all elements of the input RDD $r$ that are associated to the same key.
<li> $r.sortBy(func, asc)$. Returns an RDD where the elements of the input RDD $r$ are sorted according to the given criteria.
<li> $r.groupByKey()$. Groups the values of the input RDD $r$ association to the same key.
<li> $r.keys()$. Returns an RDD where the elements are the keys from the input RDD $r$.
<li> $r.values()$. Returns an RDD where the elements are the values from the input RDD $r$.
<li> $r.join(r1)$. Joins the values of the two RDDs $r$ and $r_1$ that have the same key.
</ol>
</font>
</p>

<hr style=" border:solid 2px;">
<p>
<font size="3" color='#91053d'><strong>Execute the following code and the read the comments in the code to understand these transformations.</strong></font>
</p>
<hr style=" border:solid 2px;">

In [16]:
'''
Create a RDD, where each item is a string s.
'''
sample_rdd = sc.parallelize(['do', 'you', 'enjoy', 'this', 'exercise', '?', 'yes', 'I', 'hope'])

'''
Transform the sample_rdd into one, where each item is (len(s), s)
'''
sample_rdd = sample_rdd.map(lambda x: (len(x), x))
print("Key-value pairs", sample_rdd.collect())

'''
Transform the previous rdd into one where items are sorted by key in descending order.
In the following instruction, x represents an item of the input RDD (a tuple (len(s), s)),
x[0] denotes the first element in the tuple (that is, len(s)).
'''

sample_rdd = sample_rdd.sortBy(lambda x: x[0], ascending=False)
print("\nOrdered key-value pairs", sample_rdd.collect())

'''
Group items by key by using
groupByKey().
The result is a RDD, where each item is (k, L), k is an integer, L is an iterable,
an object that allows one to iterate over a list.
'''
sample_rdd_gbk = sample_rdd.groupByKey()
print("\nGroup by key", sample_rdd_gbk.collect())

'''
Alternatively, grouping by key can also be achieved with the two following instructions.
The result, however, will be a RDD, where each item is (k, L), k is an integer and
L is a list (not an iterable as before).
'''
# The following instruction transforms each tuple (len(s), s) of the input RDD into a tuple
# (len(s), [s]); the second element of the tuple becomes a list.
sample_rdd_gbk_alternative = sample_rdd.map(lambda x: (x[0], [x[1]]))

#
# Given two tuples of the input RDD (len(s1), [s1]) and (len(s2), [s2]),
# in the following instruction x and y are [s1] and [s2] respectively.
# The instruction x + y concatenate the two lists
# [s1] and [s2], which gives the list [s1, s2].

sample_rdd_gbk_alternative = sample_rdd_gbk_alternative.reduceByKey(lambda x, y: x+y)
print("\nAlternative group by key", sample_rdd_gbk_alternative.collect())

'''
Obtain a RDDcontaining the keys of the RDD sample_rdd_gbk_alternative.
'''
keys = sample_rdd_gbk_alternative.keys()
print("\nKeys", keys.collect())

'''
Obtain a RDD containing the values  of the RDD sample_rdd_gbk_alternative.
'''
values = sample_rdd_gbk_alternative.values()
print("\nValues", values.collect())

Key-value pairs [(2, 'do'), (3, 'you'), (5, 'enjoy'), (4, 'this'), (8, 'exercise'), (1, '?'), (3, 'yes'), (1, 'I'), (4, 'hope')]

Ordered key-value pairs [(8, 'exercise'), (5, 'enjoy'), (4, 'this'), (4, 'hope'), (3, 'you'), (3, 'yes'), (2, 'do'), (1, '?'), (1, 'I')]

Group by key [(8, <pyspark.resultiterable.ResultIterable object at 0x7b236dd1e6e0>), (5, <pyspark.resultiterable.ResultIterable object at 0x7b2359d1b970>), (4, <pyspark.resultiterable.ResultIterable object at 0x7b2359d1a6b0>), (3, <pyspark.resultiterable.ResultIterable object at 0x7b2359d19ba0>), (2, <pyspark.resultiterable.ResultIterable object at 0x7b2359d1b8e0>), (1, <pyspark.resultiterable.ResultIterable object at 0x7b2359d1a8f0>)]

Alternative group by key [(8, ['exercise']), (5, ['enjoy']), (4, ['this', 'hope']), (3, ['you', 'yes']), (2, ['do']), (1, ['?', 'I'])]

Keys [8, 5, 4, 3, 2, 1]

Values [['exercise'], ['enjoy'], ['this', 'hope'], ['you', 'yes'], ['do'], ['?', 'I']]


### The transformation join

Let $r1$ and $r_2$ two pair RDDs.
If $(k, v1)$ and $(k, v2)$ are two elements of $r_1$ and $r_2$ respectively,
the result of $r1.join(r2)$ contains the pair $(k, (v1, v2))$.



In [17]:
r1 = sc.parallelize([('first', 1), ('second', 2), ('third', 3)])
r2 = sc.parallelize([('first', 2), ('second', 4), ('third', 9)])

rdd_join = r1.join(r2)
rdd_join.collect()

[('third', (3, 9)), ('first', (1, 2)), ('second', (2, 4))]

<hr style=" border:solid 2px;">
<p align="justify">
<font size="3">
An elegant way to write a sequence of transformations is shown in the following
code.
The tranformations are chained one after the other; each line contains a transformation.
The symbol "\" indicates that the Python instruction continues in the following line.
We recommend you to use this style in the exercises that you'll be asked to
do later.
</font>
</p>
<hr style=" border:solid 2px;">

In [18]:
'''
Create a RDD, where each item is a string s.
'''
sample_rdd = sc.parallelize(['do', 'you', 'enjoy', 'this', 'exercise', '?', 'yes', 'I', 'hope'])\
                .map(lambda x: (len(x), x))\
                .map(lambda x: (x[0], [x[1]]))\
                .reduceByKey(lambda x, y: x+y)\
                .sortBy(lambda x: x[0], ascending=False)

sample_rdd.collect()

[(8, ['exercise']),
 (5, ['enjoy']),
 (4, ['this', 'hope']),
 (3, ['you', 'yes']),
 (2, ['do']),
 (1, ['?', 'I'])]

## B.3 Actions on Pair RDDs

<p justify="align">
<font size="3">
As with the transformations, all actions available for standard RDDs can be used on Pair RDDs as well.
In addition, the following actions can be performed on Pair RDDs:
<ol>
<li> $countByKey()$. Returns a Python dictionary, where each key is mapped to the number of values associated to that key.
<li> $collectAsMap()$. Collects the RDD as a dictionary (in the same way as the function $collect()$ returns a list from a standard RDD).
<li> $lookup(key)$. Returns a list with all the values associated with the given _key_.
</ol>
</font>
</p>


# Exercises

<p align="justify">
<font size="3">
In the following exercises, you'll have to use RDD transformations/actions to implement some computations.
</font>
</p>


## 1 - Processing Text

<hr style="border:solid 2px;">

###  Exercise 1.1 - Creating an RDD


<p align="justify">
<font size="3">
Write the code to create an RDD from the input text file './data/moby-dick.txt' .
</font>
</p>
<hr style="border:solid 2px;">

In [19]:
'''
Loads a text file in RDD

INPUT:
        the filename


'''
def load_text(filename):
    return sc.textFile(filename)


mobydick = load_text('./data/moby-dick.txt')
print("If the execution arrives there probably you loaded the file in a RDD")

If the execution arrives there probably you loaded the file in a RDD



###  Utility functions

Here we provide some utility functions that will be usefull to read and to pre-process a file.



*   <code> remove_non_letters(word)</code> takes as input a word and removes any character that is not a letter.
*   <code> load_stopwords(words) </code> reads a file that contains the stopwords and returns them in a <code>list</code>.



In [20]:
import re


# Regular expression for removing all non-letter characters in the file.
regex = re.compile('[^a-zA-Z ]')

'''
Removes any non-letter character from the given word.

INPUT:
        word: A word

OUTPUT:
        the input word without the non-letter characters.

'''
def remove_non_letters(word):
    return regex.sub('', word)


'''
INPUT:
        stopwords_file: name of the file containing the stopwords.
OUTPUT:
        a Python list with the stopwords read from the file.
'''
def load_stopwords(stopwords_file):
    stopwords = []
    with open(stopwords_file) as file:
        for sw in file:
            stopwords.append(sw.strip())
    return stopwords




<hr style="border:solid 2px;">

###  Exercise 1.2  - Preprocessing

<p align="justify">
<font size="3">
We want to code a function $preprocess()$ that takes in the content of a text file, filters out useless words and
returns the remaining words.
This function has the following input and output:
<ul>
 <li> INPUT: A RDD $text$, where each element is a line of a text file, and a Python list $stopwords$ that
      contains the most common English *stopwords* (e.g., 'the', 'in', 'of'), that only serve a grammatical purpose, while adding little or no meaning to the other words in the file.
<li> OUTPUT: A RDD, where each element is a word.
</ul>
</font>
</p>

<p align="justify">
<font size="3" color='#91053d'>
<strong>Complete the code of the function $preprocess()$ with the following steps:</strong>
<ol>
<li>    Split each line into its constituent words (Python has a function $split$).
<li>    Eliminate non-letter characters from each word.
<li>    Filter out empty words (words with length 0).
<li>    Lowercase all words (Python has a function $lower$).
<li>    Remove the stopwords.
</ol>
<ul>
<li> <strong>IMPORTANT:</strong> only use RDD transformations/actions, which guarantees that the computation will be distributed.
<li> At the bottom of the next cell, you'll find the expected output, so you'll know if your implementation is correct. 👍🏻
</ul>
</font>
</p>

<hr style="border:solid 2px;">

In [21]:
'''
INPUT:
        text: RDD where each element is a line of the input text file.
        stopwords: Python list containing the stopwords.
OUTPUT:
        RDD where each element is a word from the input text file.
'''
def preprocess(text, stopwords) :
    ############## WRITE YOUR CODE HERE ##############
    words = text.flatMap(lambda line: line.split(" "))\
              .map(lambda word: remove_non_letters(word))\
              .filter(lambda word: len(word) > 0).map(lambda word: word.lower())\
              .filter(lambda word: word not in stopwords)
    return words

stopwords = load_stopwords("./data/stopwords.txt")
words = preprocess(mobydick, stopwords)
words.takeOrdered(10, key = lambda x: x)

################# EXPECTED OUTPUT #################
#
# ['aback',
# 'aback',
# 'abaft',
# 'abaft',
# 'abandon',
# 'abandon',
# 'abandon',
# 'abandoned',
# 'abandoned',
# 'abandoned']
#
###################################################

['aback',
 'aback',
 'abaft',
 'abaft',
 'abandon',
 'abandon',
 'abandon',
 'abandoned',
 'abandoned',
 'abandoned']

<hr style="border:solid 2px;">

###  Exercise 1.3 - Word count

<p align="justify">
<font size="3">
We want to write the code of the function $word\_count$ that counts the number of occurrences of each word in a text file.
The function has the following input and output:
<ul>
<li> INPUT: A RDD $words$, where each element is a word from a text file $d$ (pre-processing already done).
<li> OUTPUT: A RDD, where each element is $(w, f_{w, d})$, $w$ being a word and $f_{w, d}$ being the number of times $w$ appears in $d$. The output RDD must be sorted by $f_{w, d}$ in decreasing order.
</ul>
</font>
</p>

<p align="justify">
<font size="3" color='#91053d'>
<strong>Write the code of the function $word\_count()$.</strong>
</font>
</p>

<hr style="border:solid 2px;">

In [22]:
'''
INPUT:
        words: RDD, where each element is word from the input text file (preprocessing already done!).
OUTPUT:
        RDD, where each element is (w, occ), w is a word and occ the number of occurrences of w.
        The RDD is sorted by value in decreasing order.
'''
def word_count(words):
    ############## WRITE YOUR CODE HERE ##############'''
    occs = words.map(lambda word: (word, 1))\
                .reduceByKey(lambda x, y: x+y)\
                .sortBy(lambda f: f[1], ascending=False)
    return occs

occs = word_count(words)
occs.take(5)

################# EXPECTED OUTPUT #################
#
# [('whale', 891), ('one', 875), ('old', 436), ('man', 433), ('ahab', 417)]
#
###################################################



[('whale', 891), ('one', 875), ('old', 436), ('man', 433), ('ahab', 417)]

## - 2 Bigrams

<p align="justify">
<font size="3">
What is a Bigram? In our example a bigram is a couple of
two consecutive words in a same line.
    

For example, the previous sentence contains the following bigrams: "A bigram", "bigram is", "is a", "a couple", etc.
</font>
</p>

<p align="justify">
<font size="3">
This exercise can be seen as a simple extension of the word count, take into account this while implementing your data structures and your code.
</font>
</p>



<p align="justify">
<font size="3">
As first step here is the to create a new RDD from the input text file './data/moby-dick.txt' that we already used in the first series of the exercises.
</font>
</p>
<hr style="border:solid 2px;">

<p align="justify">
<font size="3">
We provide then a possible implementation of a function that reads the document and give as output the list of bigram in the text.

<ul>
<li> INPUT: A RDD, where each element is a line from a text file $d$ (pre-processing already done).
<li> OUTPUT: A list, where each element $(w1, w2)$ is a bigram.
</ul>

For this exercise we will focus at line level: a bigram can't be on two different lines (even if we did not put a full stop before). You can add more accurate analysis of the text and of the sentences (at syntactical and semantical level) if you want to play with coding.

Feel free to re-implement your read function if you want to use a different data-structure.
</font>
</p>
<hr style="border:solid 2px;">





In [23]:
from operator import add

'''
Returns the list of bigrams in a given text for each line (the end of a bigram search space is a line
of text:
"This  sentence
contains two different lines for looking for bigrams."
INPUT:
        a line of a text

OUTPUT:
        a list of bigrams for the text of the form b1_b2 where "b1_b2" is a string.

'''

def parse_bigrams(line):
    bigrams = []
    words = line.strip().split(" ")
    for i in range(len(words) - 1):
        bigram_w1 = remove_non_letters(words[i].lower())
        bigram_w2 = remove_non_letters(words[i+1].lower())
        if (len(bigram_w1) > 0) & (len(bigram_w2) > 0) :
                bigrams.append(bigram_w1 + "_" + bigram_w2)
    return bigrams

### Exercise 2.1 - MapReduce

<p align="justify">
<font size="3">
Describe MapReduce procedure that gives as output the number of different bigrams that appear all along the document (if the bigram "is a" appears twice in the document must be counted just one time).

</font>
</p>

In [24]:
#Describe here your algorithm

### Exercise 2.2 - Implementation

<p align="justify">
<font size="3">
Propose a Spark implementation of your procedure that follows a map-reduce approach.
</font>
</p>

In [25]:
'''
Write here a procedure that

INPUT:
        an RDD containing strings representing words

OUTPUT:
        the number of distinct bigrams.

flatMap  .....

map ....

reduce ....

'''

'''############## WRITE YOUR CODE HERE ##############'''

counts = mobydick.flatMap(parse_bigrams)\
                 .map(lambda x: (x,1))\
                 .reduceByKey(lambda x,y: x+y)
output = counts.collect()


'''############## END OF THE EXERCISE ##############'''



print(counts.take(5))
print(counts.count())

################# EXPECTED OUTPUT #################
#
# [('call_me', 3), ('me_ishmael', 1), ('some_years', 1), ('years_agonever', 1), ('agonever_mind', 1)]
# 100513
###################################################

[('call_me', 3), ('me_ishmael', 1), ('some_years', 1), ('years_agonever', 1), ('agonever_mind', 1)]
100513


### Exercise 2.3 - Smart parsing of bigrams

<p align="justify">
<font size="3">
Sometimes when you write functions you do not realize that you can use a MapReduce series of steps.
Look at the provided function that parses bigrams and outputs a string representation of them. Re-think the functions using map, reduce, and filter operators,
    in Spark.
    </font>
</p>

<p align="justify">
<font size="3">
Propose a Spark implementation of the procedure  that follows a map-reduce approach, parses lines of text, and
gives as output pairs of strings representing bigrams. For this step you just can to re-use the
 functions we already studied, for example:
<ul>
    <li> map() </li>
    <li> flatMap() </li>
    <li> filter() </li>
</ul>

Notice that you can be more flexible in the structure of the
</font>
</p>

In [26]:
'''
Returns a list of pairs of strings representing bigrams in a given text.
The procedure returns bigrams for each line (the search space of the bigrams is a line of text):
"This  sentence
contains two different lines and ('sentence','contains') is not in the output."
INPUT:
        a line of a text

OUTPUT:
        a list of pairs bigrams for the text of the form (b1, b2) where ('b1', 'b2')" is a pair of strings.

'''



'''############## WRITE YOUR CODE HERE ##############'''
def parse_smart_parse_bigrams(text_rdd):
    bigrams = text_rdd.map(lambda line: line.split(" ")) \
                   .flatMap(lambda xs: (tuple(x) for x in zip(xs, xs[1:])))\
                   .map(lambda x : (remove_non_letters(x[0].lower()), remove_non_letters(x[1].lower())))\
                   .filter(lambda x : len(x[0]) > 0 and len(x[1]) > 0)
    return bigrams
'''############## END OF THE EXERCISE ##############'''



bigrams=parse_smart_parse_bigrams(mobydick)
result = bigrams.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y)

print(result.count())
print(result.take(100))

################# EXPECTED OUTPUT #################
#
#  100513
#[(('call', 'me'), 3), (('me', 'ishmael'), 1), (('some', 'years'), 1), (('years', 'agonever'), 1), (('agonever', 'mind'), 1), (('mind', 'how'), 6), (('how', 'long'), 7), (('preciselyhaving', 'little'), 1), (('little', 'or'), 6), (('or', 'no'), 6), (('no', 'money'), 1), (('money', 'in'), 3), (('in', 'my'), 46), (('my', 'purse'), 1), (('purse', 'and'), 1), (('and', 'nothing'), 6), (('particular', 'to'), 2), (('to', 'interest'), 1), (('interest', 'me'), 1), (('me', 'on'), 8), (('on', 'shore'), 3), (('shore', 'i'), 1), (('i', 'thought'), 38), (('thought', 'i'), 31), (('i', 'would'), 30), (('would', 'sail'), 1), (('sail', 'about'), 2), (('about', 'a'), 12), (('little', 'and'), 3), (('and', 'see'), 9), (('see', 'the'), 23), (('the', 'watery'), 14), (('watery', 'part'), 1), (('part', 'of'), 89), (('of', 'the'), 1726), (('the', 'world'), 68), (('it', 'is'), 263), (('is', 'a'), 109), (('a', 'way'), 4), (('way', 'i'), 1), (('i', 'have'), 107), (('have', 'of'), 2), (('driving', 'off'), 1), (('off', 'the'), 31), (('the', 'spleen'), 1), (('spleen', 'and'), 1), (('and', 'regulating'), 1), (('regulating', 'the'), 1), (('the', 'circulation'), 1), (('whenever', 'i'), 4), (('find', 'myself'), 2), (('myself', 'growing'), 1), (('growing', 'grim'), 1), (('grim', 'about'), 1), (('about', 'the'), 55), (('the', 'mouth'), 10), (('mouth', 'whenever'), 1), (('whenever', 'it'), 2), (('a', 'damp'), 1), (('drizzly', 'november'), 1), (('november', 'in'), 1), (('my', 'soul'), 16), (('soul', 'whenever'), 1), (('i', 'find'), 3), (('myself', 'involuntarily'), 1), (('pausing', 'before'), 2), (('before', 'coffin'), 1), (('coffin', 'warehouses'), 1), (('warehouses', 'and'), 1), (('and', 'bringing'), 3), (('bringing', 'up'), 2), (('up', 'the'), 47), (('the', 'rear'), 10), (('rear', 'of'), 2), (('of', 'every'), 10), (('funeral', 'i'), 1), (('i', 'meet'), 2), (('meet', 'and'), 1), (('and', 'especially'), 10), (('especially', 'whenever'), 1), (('whenever', 'my'), 1), (('my', 'hypos'), 1), (('hypos', 'get'), 1), (('get', 'such'), 1), (('such', 'an'), 31), (('an', 'upper'), 1), (('hand', 'of'), 6), (('of', 'me'), 13), (('me', 'that'), 36), (('that', 'it'), 49), (('it', 'requires'), 2), (('requires', 'a'), 2), (('a', 'strong'), 8), (('strong', 'moral'), 1), (('moral', 'principle'), 1), (('principle', 'to'), 1), (('to', 'prevent'), 4), (('prevent', 'me'), 1), (('from', 'deliberately'), 1), (('deliberately', 'stepping'), 1)]
###################################################

100513
[(('call', 'me'), 3), (('me', 'ishmael'), 1), (('some', 'years'), 1), (('years', 'agonever'), 1), (('agonever', 'mind'), 1), (('mind', 'how'), 6), (('how', 'long'), 7), (('preciselyhaving', 'little'), 1), (('little', 'or'), 6), (('or', 'no'), 6), (('no', 'money'), 1), (('money', 'in'), 3), (('in', 'my'), 46), (('my', 'purse'), 1), (('purse', 'and'), 1), (('and', 'nothing'), 6), (('particular', 'to'), 2), (('to', 'interest'), 1), (('interest', 'me'), 1), (('me', 'on'), 8), (('on', 'shore'), 3), (('shore', 'i'), 1), (('i', 'thought'), 38), (('thought', 'i'), 31), (('i', 'would'), 30), (('would', 'sail'), 1), (('sail', 'about'), 2), (('about', 'a'), 12), (('little', 'and'), 3), (('and', 'see'), 9), (('see', 'the'), 23), (('the', 'watery'), 14), (('watery', 'part'), 1), (('part', 'of'), 89), (('of', 'the'), 1726), (('the', 'world'), 68), (('it', 'is'), 263), (('is', 'a'), 109), (('a', 'way'), 4), (('way', 'i'), 1), (('i', 'have'), 107), (('have', 'of'), 2), (('driving', 'off'), 1), 

### Exercise 2.4 - Bigrams search

<p align="justify">
<font size="3">
Look at the two procedures that parse bigrams and provide the output in two different format: is the type of the
   key influencing the map-reduce procedure? </font>
</p>

<p align="justify">
<font size="3">
From the performances point of view which data representation is better according to your opinion? Explain and motivate your answer
</font>
</p>

In [27]:
#Also in distributed context you have to consider:
#1) computation performance
#2) space issues
#3) data representation
#4) code mantainance
#5) cluster configuration
#and think and run experiment about the better solution playing with spark configuration, data representation and code.


### Exercise 2.5 - Most common bigrams

<p align="justify">
<font size="3">
Now that we start being confident with bigrams and MapReduce, we can start analysing the bigrams and write a procedure that gives as output the top 5 most common bigrams that appear all along the document.
</font>
</p>

<p align="justify">
<font size="3">
Propose a Spark implementation of your procedure that transfomrs the key-value pairs and uses the top operation
</font>
</p>

In [28]:
'''############## WRITE YOUR CODE HERE ##############'''
inverted_result = result.map(lambda a: (a[1],a[0]))
rw_result = bigrams.map(lambda x: (x, 1)).reduceByKey(lambda x, y: x + y).sortBy(lambda f: f[1], ascending=False)
ri_result = result.takeOrdered ( 10, key = lambda x : -x[1] )
'''############## END OF THE EXERCISE ##############'''

print(rw_result.take(5))
print(inverted_result.top(5))




################# EXPECTED OUTPUT #################
#
# [(1726, ('of', 'the')), (1078, ('in', 'the')), (674, ('to', 'the')), (404, ('from', 'the')), (348, ('and', 'the'))]
###################################################

[(('of', 'the'), 1726), (('in', 'the'), 1078), (('to', 'the'), 674), (('from', 'the'), 404), (('and', 'the'), 348)]
[(1726, ('of', 'the')), (1078, ('in', 'the')), (674, ('to', 'the')), (404, ('from', 'the')), (348, ('and', 'the'))]


### Exercise 2.6 - Unique bigrams

<p align="justify">
<font size="3">
Try to re-use some of the steps you implemented in the previous exercise and provide the functions that counts the bigrams that appear only once in the text (if the bigram "is a" appears twice in the text it must not be counted in the final result).
</font>
</p>

In [29]:
'''############## WRITE YOUR CODE HERE ##############'''


unique_bigrams = inverted_result.filter(lambda a : a[0] ==1)


'''############## END OF THE EXERCISE ##############'''


print(unique_bigrams.take(5))

################# EXPECTED OUTPUT #################
#
#[(1, ('me', 'ishmael')), (1, ('preciselyhaving', 'little')), (1, ('no', 'money')), (1, ('way', 'i')), (1, ('driving', 'off'))]###################################################

[(1, ('me', 'ishmael')), (1, ('some', 'years')), (1, ('years', 'agonever')), (1, ('agonever', 'mind')), (1, ('preciselyhaving', 'little'))]
